In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Deep Learning set para determinar se a imagem é de um bebê ou de um boneco.

Primeiro, checar se a última versão está instalada

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search #instalar ou checar última versão

In [ ]:
os.listdir('/kaggle/input')

## Passo 1: baixar as imagens para verificação
Utilizaremos o duckduckgo ao invés do google para evitar bugs.

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

#função para buscar imagens 'search_images'
def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image') 

É interessante realizar uma busca para ver se o que está sendo executado condiz com o que se pretende.

In [ ]:
# se der erro no arquivo JSON, repetir até dar certo
urls = search_images('bebê humano photo', max_images=1)
urls[0]

Agora, baixaremos o arquivo:

In [ ]:
from fastdownload import download_url
dest = 'bb.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Faremos o mesmo, e um só bloco, com um exemplo de boneco:

In [ ]:
# se der erro no arquivo JSON, repetir até dar certo
urls = search_images('boneca bebe imagem', max_images=1)

dest = 'boneca.jpg'
download_url(urls[0], dest, show_progress=False)

im = Image.open(dest)
im.to_thumb(256,256)

## Passo 2: Set de imagens
Agora, vamos baixar um set de imagens, 'store' e ver um 'batch'.

In [ ]:
searches = 'bebê humano','boneca baby'
path = Path('bb_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(4)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} at home photo'))
    sleep(4)
    resize_images(path/o, max_size=200, dest=path/o)

Limpe as imagens corrompidas:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

## Passo 3: Definir bloco de dados (DataBlock)
Vamos definir o bloco de imagens a ser treinado com suas funções e, ao final, retirar um 'batch'.

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

## Passo 4: Learner (treinar o set)
Na sequência, treinamos o set de imagens e refinamos com 'fine_tune()'

In [ ]:
# pode aparecer uma mensagem de download no Kaggle
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(3)

Chegou o momento de usar o modelo (pre-deploy)

In [ ]:
is_bb,_,probs = learn.predict(PILImage.create(Path('/kaggle/input/images/titi.jpg')))
print(f"É um: {is_bb}.")
print(f"Probabilidade: {probs[0]:.2f}")
Image.open((Path('/kaggle/input/images/titi.jpg'))).to_thumb(400,400) # essa linha serve apenas para mostrar o arquivo utilizado no modelo